#### Solution 1: Manual descent

In [113]:
import math
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.transform import Rotation
from main import *
%matplotlib qt
#%matplotlib inline

ngas, nliq, n, rxgas, rygas, rzgas, rxliq, ryliq, rzliq, ex, ey, ez, emx, emy, emz, boxliq, boxgas, cutgas2, cutliq2 = initia()

C1=torch.tensor([rxgas[0], rygas[0], rzgas[0]])*boxgas
C2=torch.tensor([rxgas[1], rygas[1], rzgas[1]])*boxgas

max_angle = math.pi**2
max_radius = xl/2

e1 = [ex[0], ey[0], ez[0]]
e2 = [ex[1], ey[1], ez[1]]

em1 = [emx[0], emy[0], emz[0]]
em2 = [emx[1], emy[1], emz[1]]

params = (torch.tensor([1.])*max_angle).requires_grad_(), (torch.rand(1)*max_angle).requires_grad_(), (torch.tensor([1.])*max_angle).requires_grad_(), (torch.rand(1)*max_angle).requires_grad_()

def mincontactdistance(radius1, theta1, radius2, theta2):
    radius1=(radius1/max_angle)*max_radius
    radius2=(radius2/max_angle)*max_radius
    radius1_clamped = torch.clamp(radius1, 0, max_radius)
    radius2_clamped = torch.clamp(radius2, 0, max_radius)
    
    a1 = F.normalize(torch.tensor(em1, dtype=torch.float32), dim=0)
    n1 = F.normalize(torch.tensor(e1, dtype=torch.float32), dim=0)

    v1=F.normalize(torch.cross(n1, a1), dim=0)
    u1=F.normalize(torch.cross(n1, v1), dim=0)

    point1=C1+(radius1_clamped*torch.sin(theta1)*v1+radius1_clamped*torch.cos(theta1)*u1)

    a2 = F.normalize(torch.tensor(em2, dtype=torch.float32), dim=0)
    n2 = F.normalize(torch.tensor(e2, dtype=torch.float32), dim=0)

    v2=F.normalize(torch.cross(n2, a2), dim=0)
    u2=F.normalize(torch.cross(n2, v2), dim=0)

    point2=C2+(radius2_clamped*torch.sin(theta2)*v2+radius2_clamped*torch.cos(theta2)*u2)

    loss = (torch.sum((point2-point1)**2))**0.5
    loss.backward()
    return loss, point1, point2

def loop(params):
    iterations=200
    alpha = 0.2
    d_alpha = (1-alpha)/iterations
    for i in range(0, iterations):
        lr=0.1
        l, point1, point2 = mincontactdistance(*params)
        for p in params:
            p.data -= p.grad*lr
            p.grad.zero_()
        alpha += d_alpha
        if i == iterations-1:
            ax.scatter(*(point1.clone().detach().numpy()), color='k', alpha=1)
            ax.scatter(*(point2.clone().detach().numpy()), color='k', alpha=1)
        elif i % 20 == 0:
            print('Loss Train Set:', l.item())
            ax.scatter(*(point1.clone().detach().numpy()), color='r', alpha=alpha)
            ax.scatter(*(point2.clone().detach().numpy()), color='g', alpha=alpha)

        add_disk(e1, C1.clone().detach().numpy(), max_radius, 'r')
        add_disk(e2, C2.clone().detach().numpy(), max_radius, 'g')


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
loop(params)
ax.set_xlim([-.5*boxgas, .5*boxgas])
ax.set_ylim([-.5*boxgas, .5*boxgas])
ax.set_zlim([-.5*boxgas, .5*boxgas])
plt.show()

Loss Train Set: 17.001951217651367
Loss Train Set: 16.79012107849121
Loss Train Set: 16.773204803466797
Loss Train Set: 16.77164649963379
Loss Train Set: 16.771440505981445
Loss Train Set: 16.771404266357422
Loss Train Set: 16.77139663696289
Loss Train Set: 16.771392822265625
Loss Train Set: 16.77139663696289
Loss Train Set: 16.771390914916992


#### Solution 2: Uses optimizer

In [114]:
import math
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.transform import Rotation
from main import *
%matplotlib qt
#%matplotlib inline

ngas, nliq, n, rxgas, rygas, rzgas, rxliq, ryliq, rzliq, ex, ey, ez, emx, emy, emz, boxliq, boxgas, cutgas2, cutliq2 = initia()

C1=torch.tensor([rxgas[0], rygas[0], rzgas[0]])*boxgas
C2=torch.tensor([rxgas[1], rygas[1], rzgas[1]])*boxgas

max_angle = math.pi**2
max_radius = xl/2

e1 = [ex[0], ey[0], ez[0]]
e2 = [ex[1], ey[1], ez[1]]

em1 = [emx[0], emy[0], emz[0]]
em2 = [emx[1], emy[1], emz[1]]

params = (torch.tensor([1.])*max_angle).requires_grad_(), (torch.rand(1)*max_angle).requires_grad_(), (torch.tensor([1.])*max_angle).requires_grad_(), (torch.rand(1)*max_angle).requires_grad_()

def mincontactdistance(radius1, theta1, radius2, theta2):
    radius1=(radius1/max_angle)*max_radius
    radius2=(radius2/max_angle)*max_radius
    radius1_clamped = torch.clamp(radius1, 0, max_radius)
    radius2_clamped = torch.clamp(radius2, 0, max_radius)
    
    a1 = F.normalize(torch.tensor(em1, dtype=torch.float32), dim=0)
    n1 = F.normalize(torch.tensor(e1, dtype=torch.float32), dim=0)

    v1=F.normalize(torch.cross(n1, a1), dim=0)
    u1=F.normalize(torch.cross(n1, v1), dim=0)

    point1=C1+(radius1_clamped*torch.sin(theta1)*v1+radius1_clamped*torch.cos(theta1)*u1)

    a2 = F.normalize(torch.tensor(em2, dtype=torch.float32), dim=0)
    n2 = F.normalize(torch.tensor(e2, dtype=torch.float32), dim=0)

    v2=F.normalize(torch.cross(n2, a2), dim=0)
    u2=F.normalize(torch.cross(n2, v2), dim=0)

    point2=C2+(radius2_clamped*torch.sin(theta2)*v2+radius2_clamped*torch.cos(theta2)*u2)

    loss = (torch.sum((point2-point1)**2))**0.5
    loss.backward()
    return loss, point1, point2

def loop(params):
    optimizer = torch.optim.Adam([*params], lr=0.05, weight_decay=0)

    iterations = 70

    alpha = 0.2
    d_alpha = (1-alpha)/iterations
    for i in range(0, iterations):
        optimizer.zero_grad()
        l, point1, point2 = mincontactdistance(*params)
        optimizer.step()
        alpha += d_alpha
        if i == iterations-1:
            ax.scatter(*(point1.clone().detach().numpy()), color='k', alpha=1)
            ax.scatter(*(point2.clone().detach().numpy()), color='k', alpha=1)
        elif i % 10 == 0:
            print('Loss Train Set:', l.item())
            ax.scatter(*(point1.clone().detach().numpy()), color='r', alpha=alpha)
            ax.scatter(*(point2.clone().detach().numpy()), color='g', alpha=alpha)

    print('Loss Train Set:', l.item())

    add_disk(e1, C1.clone().detach().numpy(), max_radius, 'r')
    add_disk(e2, C2.clone().detach().numpy(), max_radius, 'g')
    
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
loop(params)
ax.set_xlim([-.5*boxgas, .5*boxgas])
ax.set_ylim([-.5*boxgas, .5*boxgas])
ax.set_zlim([-.5*boxgas, .5*boxgas])
plt.show()

Loss Train Set: 18.20840072631836
Loss Train Set: 17.976947784423828
Loss Train Set: 17.695390701293945
Loss Train Set: 17.357221603393555
Loss Train Set: 17.054887771606445
Loss Train Set: 16.875017166137695
Loss Train Set: 16.804046630859375
Loss Train Set: 16.7760009765625
